In [3]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import yfinance as yf
import yahoo_fin.options as ops
from yahoo_fin.stock_info import get_data


In [7]:

# Load your stock data
# Assuming you have a DataFrame with columns: 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'
# Adjust the code according to your data structure
def get_ticker_data(ticker):
    df = get_data(ticker, start_date="01/01/2022")

    df.reset_index(inplace=True)
    df.rename(columns={'index': 'date'}, inplace=True)

    return df

your_data = get_ticker_data('META')

print(your_data.head())


        date        open        high         low       close    adjclose  \
0 2022-01-03  338.299988  341.079987  337.190002  338.540009  338.181183   
1 2022-01-04  339.950012  343.089996  331.869995  336.529999  336.173309   
2 2022-01-05  333.019989  335.760010  323.839996  324.170013  323.826416   
3 2022-01-06  322.820007  339.170013  322.720001  332.459991  332.107635   
4 2022-01-07  332.739990  337.000000  328.880005  331.790009  331.438354   

     volume ticker  
0  14537900   META  
1  15998000   META  
2  20564500   META  
3  27962800   META  
4  14722000   META  


In [18]:

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_data = scaler.fit_transform(your_data[['open', 'high', 'low', 'close', 'volume']])
scaled_data = scaler.fit_transform(your_data[['open', 'high', 'low', 'close']])

# Define function to create dataset
def create_dataset(data, look_back=1):
    X, Y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), :])
        Y.append(data[i + look_back, 3])  # Assuming you're predicting 'Close' price
    # print(X,Y)
    return np.array(X), np.array(Y)


In [20]:

# Split data into train and test sets
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size
train_data, test_data = scaled_data[0:train_size, :], scaled_data[train_size:len(scaled_data), :]

# Create dataset for LSTM
look_back = 7  # Number of previous days to use as input features
train_X, train_Y = create_dataset(train_data, look_back)
test_X, test_Y = create_dataset(test_data, look_back)

# Reshape input data to be 3D [samples, time steps, features]
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 4))
test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[1], 4))


In [24]:

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(look_back, 4)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')


In [25]:

# Train the model
model.fit(train_X, train_Y, epochs=100, batch_size=32)

# Make predictions
train_predict = model.predict(train_X)
test_predict = model.predict(test_X)

# Inverse scaling to get actual prices
train_predict = scaler.inverse_transform(np.concatenate((np.zeros((len(train_predict), 3)), train_predict), axis=1))[:, 3]
test_predict = scaler.inverse_transform(np.concatenate((np.zeros((len(test_predict), 3)), test_predict), axis=1))[:, 3]
train_Y = scaler.inverse_transform(np.concatenate((np.zeros((len(train_Y), 3)), train_Y.reshape(-1, 1)), axis=1))[:, 3]
test_Y = scaler.inverse_transform(np.concatenate((np.zeros((len(test_Y), 3)), test_Y.reshape(-1, 1)), axis=1))[:, 3]

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(train_Y, train_predict))
test_rmse = np.sqrt(mean_squared_error(test_Y, test_predict))
print('Train RMSE: %.2f' % train_rmse)
print('Test RMSE: %.2f' % test_rmse)

print(test_Y[-1])
print(test_predict)

Epoch 1/100


15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0165
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0023 
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0010 
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.1191e-04 
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.2056e-04
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.0202e-04 
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.1514e-04 
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.0480e-04 
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.3068e-04 
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.5067e-04 
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.2464e-04 
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.5454e-04 
Epoch 13/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.1484e-04
Epoch 14/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6.2931e-04 
Epoch 15/100
15/15 ━━━━━━━━━